In this section, we cover:
- Accessing parameters for debugging, diagnostics, and visualizations
- Sharing parameters across different model componetns

In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(
    nn.LazyLinear(8),
    nn.ReLU(),
    nn.LazyLinear(1)
)

X = torch.rand(size=(2,4))
net(X).shape

/opt/miniconda3/envs/d2l/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

When w model is defined via the Sequential class, we can first access any layer by indexing into the model as though it were a list. Each layer's parameters are conveniently located in its attribute.

We can inspect the parameters of the second fully connected layer as follows.

In [6]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1851, -0.0598,  0.1091, -0.0429, -0.0689,  0.2480, -0.1503, -0.1469]])),
             ('bias', tensor([0.0506]))])

We can see that this fully connected lauer contains two parameters, corresponding to that layer's weights and biases, respectively.

Note that each parameter is represented as an instance of the parameter class. To do anything useful with parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter's value.

In [7]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0506]))

Parameters are complex objects, containing values, gradients, and additional information. That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

In [8]:
net[2].weight.grad == None

True

When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unweildy when we work with more complex, e.g., nested, modules, since we would need to recurse through the entire tree to extract each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.

In [9]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

Often, we want to share parameters across multiple layers. Let's see how to do this elegantly. In the following we allocate a fully connected layer and then use its parameters specifically to set those of another layer. Here we need to run the forward propagation net(X) before accessing the parameters.

In [10]:
# We need to give the shared layer a name so that we can refer to its parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Chekc whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] == 100
# Make sure theat they are actually the same object rather than just having the same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


/opt/miniconda3/envs/d2l/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


This example shows that the parameters of the second and third layer are tied. They are not just equal, they are represented by the exact same tensor. Thus, if we change one of the parameters, the other one changes too.

You might wonder, when parameters are tied, what happens to the gradients? Since the model parameters contain gradients, the gradients of the second hidden layer and the third hidden layer are added together during backpropagation.